In [1]:
import pandas as pd

In [7]:
# Load csv
teamFile = pd.read_csv('data/NBAPlayerAges.csv')

teamFile.drop(columns = 'Unnamed: 0', inplace=True)

In [8]:
# Filter on season and team abbrevs
home_team = teamFile[(teamFile['season'] == '2019-20') & (teamFile['team_abbreviation'] == 'BOS')]
visiting_team = teamFile[(teamFile['season'] == '2019-20') & (teamFile['team_abbreviation'] == 'ATL')]
print(home_team)
print(visiting_team)

               player_name team_abbreviation  age  player_height  \
10644       Gordon Hayward               BOS   29         200.66   
10648       Grant Williams               BOS   21         198.12   
10682          Enes Kanter               BOS   27         208.28   
10696         Jayson Tatum               BOS   22         203.20   
10707         Jaylen Brown               BOS   23         198.12   
10724        Javonte Green               BOS   26         193.04   
10765       Brad Wanamaker               BOS   30         190.50   
10786       Carsen Edwards               BOS   21         180.34   
10882         Daniel Theis               BOS   27         203.20   
10898         Semi Ojeleye               BOS   25         198.12   
10919       Romeo Langford               BOS   20         193.04   
10946  Robert Williams III               BOS   22         203.20   
10950           Tacko Fall               BOS   24         226.06   
10961      Vincent Poirier               BOS   2

In [13]:
homeTeamAvg = home_team.mean()
visitTeamAvg = visiting_team.mean()

print(homeTeamAvg)
print('--------------------')
print(visitTeamAvg)

age               24.705882
player_height    197.671765
player_weight    101.978154
gp                41.000000
pts                8.688235
reb                3.647059
ast                1.741176
net_rating         5.411765
oreb_pct           0.044000
dreb_pct           0.149294
usg_pct            0.179529
ts_pct             0.560882
ast_pct            0.115059
dtype: float64
--------------------
age               25.388889
player_height    200.236667
player_weight     98.379065
gp                42.500000
pts                8.933333
reb                4.122222
ast                1.900000
net_rating        -5.288889
oreb_pct           0.045833
dreb_pct           0.136889
usg_pct            0.177944
ts_pct             0.526056
ast_pct            0.126778
dtype: float64


In [11]:
homeHeightAvg = homeTeamAvg['player_height']
homeWeightAvg = homeTeamAvg['player_weight']
homeAgeAvg = homeTeamAvg['age']
visHeightAvg = visitTeamAvg['player_height']
visWeightAvg = visitTeamAvg['player_weight']
visAgeAvg = visitTeamAvg['age']

dictRow = {'homeTeamHeightAverage': homeHeightAvg, 'homeTeamWeightAverage': homeWeightAvg,
               'homeTeamAgeAverage': homeAgeAvg, 'visitorTeamHeightAverage': visHeightAvg,
               'visitorTeamWeightAverage': visWeightAvg, 'visitorTeamAgeAverage': visAgeAvg}

dictRow

{'homeTeamHeightAverage': 197.67176470588237,
 'homeTeamWeightAverage': 101.97815435294119,
 'homeTeamAgeAverage': 24.705882352941178,
 'visitorTeamHeightAverage': 200.23666666666665,
 'visitorTeamWeightAverage': 98.37906488888888,
 'visitorTeamAgeAverage': 25.38888888888889}

In [22]:
# Turn this into data frame
predict_this_df = pd.DataFrame([dictRow])

predict_this_df

,homeTeamHeightAverage,homeTeamWeightAverage,homeTeamAgeAverage,visitorTeamHeightAverage,visitorTeamWeightAverage,visitorTeamAgeAverage
0,197.671765,101.978154,24.705882,200.236667,98.379065,25.388889


In [44]:
data_row = predict_this_df.iloc[0].values
reshape_datarow = data_row.reshape(1, -1)

print(data_row)
print(reshape_datarow)

[197.67176471 101.97815435  24.70588235 200.23666667  98.37906489
  25.38888889]
[[197.67176471 101.97815435  24.70588235 200.23666667  98.37906489
   25.38888889]]


In [29]:
# Load Model

from tensorflow.keras.models import load_model 

model = load_model('saved_deep_neural_game.h5')

# Load scaler from the model
from pickle import load

scaler = load(open('model/scaler.pkl', 'rb'))

In [52]:
# transform the test dataset
X_test_scaled = scaler.transform(reshape_datarow)

# Evaluate the model using the training data
predictions = model.predict(X_test_scaled)
predictions_round = model.predict(X_test_scaled).round()

print(predictions)
print(predictions_round) # [[Lose, Win]]

[[0.46005082 0.5399492 ]]
[[0. 1.]]


In [ ]:
# if first index is 0, home team wins
# if first index is 1, home team loss
homeTeamOutcome = predictions_round[0][0] 

if homeTeamOutcome == 0.0:
    winner = 'BOS'
else:
    winner = 'ATL'

# Gives the higher probability as output
probability_first = predictions[0][0]
probability_second = predictions[0][1]

if probability_first < probability_second:
    WinProbability = probability_second
else:
    WinProbability = probability_first